In [ ]:
# Python version 3.7.0
# Example version 1.1.2
import os
from modeller import *
from modeller.automodel import *    # Load the automodel class
from mutator_functions import *

In [ ]:
# Converts alignment file data into easy to use structure
# Specify, as for modeller, the alignment file name, knowns and target sequence
alnfile = 'picchuEAAAAE5SGS'
knowns = ('picchu1', 'EAAAAE5SGSfhnge1', 'picchuR1')
sequence= 'picchuEAAAAE5SGS1'
structure = get_structure(alnfile, [*knowns, sequence])
print("Structure obtained")

In [ ]:
# Setup your mutations here
# Below is an example of a replace mutation
linenum = 2 # Line of the target on which to perform mutation
original = 'SGS' # This is the sequence to be replaced

for i in range(2, 6):
    if i == 0:
        mutation = 'GGAS' # This is the sequence that will replace 'original'
    else:
        mutation = 'SGS'*i
    alnfile_i = alnfile + str(i) # Assigning unique filenames to each alignment file
    maximum = 1 # Maximum number of replacements to perform
    origin = 0 # Replacements will happen around character 20
    structure_m = replace_mutation(structure, linenum, original=original, 
                                 mutation=mutation, maximum=maximum, origin=origin) # Performing mutation
    structure_to_file(structure_m, alnfile_i, '.ali') # Saving mutated data into alignment file
    
    log.verbose()
    env = environ(rand_seed=-12312)  # To get different models from another script

    # directories for input atom files
    env.io.atom_files_directory = ['.','../atom_files']
    # Read in HETATM records from template PDBs
    env.io.hetatm = env.io.water = True


    a = automodel(env,
                  alnfile=alnfile_i + '.ali',      # alignment filename
                  knowns=knowns,    # codes of the templates
                  sequence=sequence,           # code of the target
                  assess_methods=assess.GA341)  # request GA341 assessment

    a.starting_model= 1                 # index of the first model
    a.ending_model  = 2                 # index of the last model
                                        # (determines how many models to calculate)
    a.deviation = 4.0                   # has to >0 if more than 1 model

    a.make()                            # do homology modelling

    # Renaming the pdbs so that the next loop doesnt overwrite them
    for id2 in range(a.starting_model, a.ending_model+1):
        output_name = a.get_model_filename(sequence=sequence, id1=9999, id2=id2, file_ext='.pdb')
        os.rename(output_name, output_name[0:-4] + str(i) + '.pdb')